### Setting up this document

In [1]:
#importing libraries
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working 

host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Suchottv20!"

src_dbname = "chinook"
dst_dbname = "chinook_dw"

In [3]:
# Define Functions for Getting Data From and Setting Data Into Databases

def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [4]:
# Create the New Data Warehouse database, and to Use it, Switch the Connection Context.

conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

### Creating and populating the dimension tables

#### Extracting data from database

In [5]:
sql_customer = "SELECT * FROM chinook.customer;"
df_customer = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customer)
df_customer.head(2)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5


In [6]:
sql_employee = "SELECT * FROM chinook.employee;"
df_employee = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employee)
df_employee.head(2)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


In [7]:
sql_track = "SELECT * FROM chinook.track;"
df_track = get_dataframe(user_id, pwd, host_name, src_dbname, sql_track)
df_track.head(2)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99


In [8]:
sql_album = "SELECT * FROM chinook.album;"
df_album = get_dataframe(user_id, pwd, host_name, src_dbname, sql_album)
df_album.head(2)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2


#### Performing transformations

In [9]:
drop_cols = ['Phone','Fax','Email','SupportRepId']
df_customer.drop(drop_cols, axis=1, inplace=True)
df_customer.rename(columns={"CustomerId":"customer_key"}, inplace=True)

df_customer.head(2)

,customer_key,FirstName,LastName,Company,Address,City,State,Country,PostalCode
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174


In [10]:
drop_cols = ['Phone','Fax','Email','ReportsTo']
df_employee.drop(drop_cols, axis=1, inplace=True)
df_employee.rename(columns={"EmployeeId":"employee_key"}, inplace=True)

df_employee.head(2)

,employee_key,LastName,FirstName,Title,BirthDate,HireDate,Address,City,State,Country,PostalCode
0,1,Adams,Andrew,General Manager,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1
1,2,Edwards,Nancy,Sales Manager,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3


In [11]:
drop_cols = ['Milliseconds','Bytes']
df_track.drop(drop_cols, axis=1, inplace=True)
df_track.rename(columns={"TrackId":"track_key"}, inplace=True)

df_track.head(2)

,track_key,Name,AlbumId,MediaTypeId,GenreId,Composer,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",0.99
1,2,Balls to the Wall,2,2,1,None,0.99


In [12]:
df_album.rename(columns={"AlbumId":"album_key"}, inplace=True)

df_album.head(2)

,album_key,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2


#### Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

In [13]:
db_operation = "insert"

tables = [('dim_customer', df_customer, 'customer_key'),
          ('dim_employee', df_employee, 'employee_key'),
          ('dim_track', df_track, 'track_key'),
          ('dim_album', df_album, 'album_key')]

In [14]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

### Create and populate fact table

#### Getting data from all relavent tables

In [29]:
sql_invoice = "SELECT * FROM Chinook.invoice;"
df_invoice = get_dataframe(user_id, pwd, host_name, src_dbname, sql_invoice)
df_invoice.rename(columns={"InvoiceId":"invoice_id"}, inplace=True)
df_invoice.head(2)

,invoice_id,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96


In [30]:
sql_invoiceline = "SELECT * FROM Chinook.invoiceline;"
df_invoiceline = get_dataframe(user_id, pwd, host_name, src_dbname, sql_invoiceline)
df_invoiceline.rename(columns={"InvoiceLineId":"invoiceline_id"}, inplace=True)
df_invoiceline.rename(columns={"InvoiceId":"invoice_id"}, inplace=True)
df_invoiceline.head(2)

,invoiceline_id,invoice_id,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1


#### Join the two above tables to create Fact Table.

In [31]:
df_invoice = pd.merge(df_invoice, df_invoiceline, on='invoice_id', how='right')
df_invoice.drop(['invoice_id'], axis=1, inplace=True)
df_invoice.head(4)

,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,invoiceline_id,TrackId,UnitPrice,Quantity
0,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,1,2,0.99,1
1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,4,0.99,1
2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,3,6,0.99,1
3,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,8,0.99,1


##### Perform any Additional Transformations


In [32]:
# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_invoice.insert(0, "order_key", range(1, df_invoice.shape[0]+1))
df_invoice.head(5)

,order_key,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,invoiceline_id,TrackId,UnitPrice,Quantity
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,1,2,0.99,1
1,2,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,4,0.99,1
2,3,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,3,6,0.99,1
3,4,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,8,0.99,1
4,5,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,5,10,0.99,1


##### Write the DataFrame Back to the Database

In [33]:
table_name = "fact_invoice"
primary_key = "order_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_invoice, table_name, primary_key, db_operation)

### Demonstrate that the New Data Warehouse Exists and Contains the Correct Data


In [36]:
sql_test = """
    SELECT CustomerId AS `customer_id`,
        SUM(Quantity) AS `total_quantity`,
        SUM(UnitPrice) AS `total_unit_price`
    FROM `{0}`.`fact_invoice` AS invoice

    GROUP BY customer_id    
    ORDER BY total_unit_price DESC;
    
""".format(dst_dbname)

df_test = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test)

In [37]:
df_test.head(100)

,customer_id,total_quantity,total_unit_price
0,6,38,49.62
1,26,38,47.62
2,57,38,46.62
3,46,38,45.62
4,45,38,45.62
5,37,38,43.62
6,24,38,43.62
7,28,38,43.62
8,25,38,42.62
9,7,38,42.62
